In [3]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [8]:
nb_classes = 10

# input image dimensions
img_rows, img_cols = 28, 28

# load training data and do basic data normalization
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

def make_model(dense_layer_sizes, nb_filters, nb_conv, nb_pool):
    '''Creates model comprised of 2 convolutional layers followed by dense layers

    dense_layer_sizes: List of layer sizes. This list has one number for each layer
    nb_filters: Number of convolutional filters in each convolutional layer
    nb_conv: Convolutional kernel size
    nb_pool: Size of pooling area for max pooling
    '''

    model = Sequential()

    model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                            border_mode='valid',
                            input_shape=(img_rows, img_cols, 1)))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    return model

In [11]:
dense_size_candidates = [[32], [64], [32, 32], [64, 64]]
my_classifier = KerasClassifier(make_model, batch_size=32)
validator = GridSearchCV(my_classifier,
                         param_grid={'dense_layer_sizes': dense_size_candidates,
                                     # nb_epoch is avail for tuning even when not
                                     # an argument to model building function
                                     'nb_epoch': [3, 6],
                                     'nb_filters': [8],
                                     'nb_conv': [3],
                                     'nb_pool': [2]},
                         scoring='neg_log_loss',
                         n_jobs=1)

In [12]:
validator.fit(X_train, y_train)

Epoch 1/3
40000/40000 [==============================] - 9s - loss: 0.8562 - acc: 0.7066     
Epoch 2/3
40000/40000 [==============================] - 8s - loss: 0.5570 - acc: 0.8139     
Epoch 3/3
40000/40000 [==============================] - 9s - loss: 0.8571 - acc: 0.7047     
Epoch 2/3
40000/40000 [==============================] - 8s - loss: 0.5768 - acc: 0.8102     
Epoch 3/3
40000/40000 [==============================] - 9s - loss: 0.8188 - acc: 0.7274     
Epoch 2/3
40000/40000 [==============================] - 8s - loss: 0.5018 - acc: 0.8389     
Epoch 3/3
40000/40000 [==============================] - 9s - loss: 0.8991 - acc: 0.6965     
Epoch 2/6
40000/40000 [==============================] - 8s - loss: 0.5997 - acc: 0.8055     
Epoch 3/6
40000/40000 [==============================] - 8s - loss: 0.5111 - acc: 0.8363     
Epoch 4/6
40000/40000 [==============================] - 8s - loss: 0.4527 - acc: 0.8563     
Epoch 5/6
40000/40000 [==============================] - 8s 

GridSearchCV(cv=None, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f2de647c8d0>,
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'nb_epoch': [3, 6], 'nb_filters': [8], 'nb_conv': [3], 'nb_pool': [2], 'dense_layer_sizes': [[32], [64], [32, 32], [64, 64]]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

In [13]:
print('The parameters of the best model are: ')
print(validator.best_params_)

# validator.best_estimator_ returns sklearn-wrapped version of best model.
# validator.best_estimator_.model returns the (unwrapped) keras model
best_model = validator.best_estimator_.model
metric_names = best_model.metrics_names
metric_values = best_model.evaluate(X_test, y_test)
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)

The parameters of the best model are: 
{'nb_epoch': 6, 'nb_filters': 8, 'nb_conv': 3, 'nb_pool': 2, 'dense_layer_sizes': [64, 64]}
 9952/10000 [============================>.] - ETA: 0sloss :  0.0557825804951
acc :  0.9829
